In [2]:
# IMPORT STANDART LIBRARIES
import os
import re
import pandas as pd
from IPython.display import display

import win32con, win32api, os

# FILES:
# Root-folder path
main_path = 'Z:\\Projects\\РФФИ-Норвегия\\Данные\\Экспериментальные данные в лаборатории\\Droplet freezing facility data\\Generating droplets'

# Name of folder containing droplet properties
imgproc_folder = 'IMAGE_PROCESSING_DATA'
imgproc_path = os.path.join(main_path, imgproc_folder)

## Append all imgproc-files in one df

In [16]:
# get files with droplet properties
all_files = os.listdir(imgproc_path)
str_files = ','.join(all_files)
pixprop_files = re.findall(r'\d\d\d\d_pixprop_\d.csv',str_files)

In [19]:
# set test-range for finding
test_range = [28,300]

prop_df = pd.DataFrame()
# in setting test-range: collect data
for file_name in pixprop_files:
    # get number of test
    test_num = int(file_name.split('_')[0])
    if (test_num>=test_range[0]) and (test_num<=test_range[1]):
        print('analyzing',file_name)
        
        # get number of crop
        crop_num = int(file_name.split('_')[-1].split('.')[0])

        # read csv-file
        curr_df = pd.read_csv(os.path.join(imgproc_path,file_name))

        # add test,crop num columns
        curr_df['test'] = test_num
        curr_df['crop'] = crop_num
        # change columns order
        cols = curr_df.columns.to_list()
        new_cols = cols[-2:] + cols[:-2]
        curr_df = curr_df[new_cols]

        # add dataframes
        prop_df = prop_df.append(curr_df, ignore_index=True)
display(prop_df)

analyzing 0029_pixprop_1.csv
analyzing 0029_pixprop_2.csv
analyzing 0029_pixprop_3.csv
analyzing 0029_pixprop_4.csv
analyzing 0029_pixprop_5.csv
analyzing 0030_pixprop_1.csv
analyzing 0030_pixprop_2.csv
analyzing 0030_pixprop_3.csv
analyzing 0031_pixprop_1.csv
analyzing 0031_pixprop_2.csv
analyzing 0031_pixprop_3.csv
analyzing 0031_pixprop_4.csv
analyzing 0031_pixprop_5.csv
analyzing 0033_pixprop_1.csv
analyzing 0033_pixprop_2.csv
analyzing 0037_pixprop_1.csv
analyzing 0037_pixprop_2.csv
analyzing 0038_pixprop_1.csv
analyzing 0038_pixprop_2.csv
analyzing 0040_pixprop_1.csv
analyzing 0040_pixprop_2.csv
analyzing 0044_pixprop_1.csv
analyzing 0044_pixprop_2.csv
analyzing 0045_pixprop_1.csv
analyzing 0045_pixprop_2.csv
analyzing 0045_pixprop_3.csv
analyzing 0046_pixprop_1.csv
analyzing 0046_pixprop_2.csv
analyzing 0046_pixprop_3.csv
analyzing 0046_pixprop_4.csv
analyzing 0047_pixprop_1.csv
analyzing 0047_pixprop_2.csv
analyzing 0047_pixprop_3.csv
analyzing 0047_pixprop_4.csv
analyzing 0049

,test,crop,frameNum,y,x,diameter,area
0,29,1,16,193.530266,511.654358,45.862749,1652
1,29,1,17,196.702913,511.446602,44.352622,1545
2,29,1,19,206.938872,511.378635,46.318556,1685
3,29,1,21,219.275484,511.092903,44.424332,1550
4,29,1,22,227.751666,511.210176,45.848866,1651
...,...,...,...,...,...,...,...
8550,272,3,74,1129.120773,632.701087,45.918239,1656
8551,272,3,75,1161.783364,633.420857,45.290053,1611
8552,272,3,76,1194.867085,634.314107,45.064588,1595
8553,272,3,77,1228.344099,634.962733,45.275994,1610


In [21]:
# save dataframe
save_name = 'imgproc_detailed.csv'
prop_df.to_csv(os.path.join(imgproc_path,save_name),index=False)

## Modify and save current imgproc file

In [31]:
# get pixel_sizes & record_rates
improc_df = pd.read_csv(os.path.join(imgproc_path,'imgproc_detailed.csv'))
#display(improc_df.info())
main_df = pd.read_csv(os.path.join(main_path,'GENERATING_DATA.csv'))
#display(main_df.info())
calibr_df = main_df[['test','crop','pixel_size','record_rate']]
improc_df = improc_df.merge(calibr_df,on=['test','crop'],how='left')
improc_df.columns = ['test', 'crop', 'frameNum', 'y_pix', 'x_pix','diameter_pix','area_pix','pixel_size','record_rate']
display(improc_df)

,test,crop,frameNum,y_pix,x_pix,diameter_pix,area_pix,pixel_size,record_rate
0,29,1,16,193.530266,511.654358,45.862749,1652,24.20,1000.0
1,29,1,17,196.702913,511.446602,44.352622,1545,24.20,1000.0
2,29,1,19,206.938872,511.378635,46.318556,1685,24.20,1000.0
3,29,1,21,219.275484,511.092903,44.424332,1550,24.20,1000.0
4,29,1,22,227.751666,511.210176,45.848866,1651,24.20,1000.0
...,...,...,...,...,...,...,...,...,...
8550,272,3,74,1129.120773,632.701087,45.918239,1656,21.61,1000.0
8551,272,3,75,1161.783364,633.420857,45.290053,1611,21.61,1000.0
8552,272,3,76,1194.867085,634.314107,45.064588,1595,21.61,1000.0
8553,272,3,77,1228.344099,634.962733,45.275994,1610,21.61,1000.0


In [32]:
# time in seconds
improc_df['time'] = improc_df['frameNum']/improc_df['record_rate']
# coordinates in mm
improc_df['y'] = improc_df['y_pix']*improc_df['pixel_size']*1e-3
improc_df['x'] = improc_df['x_pix']*improc_df['pixel_size']*1e-3
display(improc_df)

,test,crop,frameNum,y_pix,x_pix,diameter_pix,area_pix,pixel_size,record_rate,time,y,x
0,29,1,16,193.530266,511.654358,45.862749,1652,24.20,1000.0,0.016,4.683432,12.382035
1,29,1,17,196.702913,511.446602,44.352622,1545,24.20,1000.0,0.017,4.760210,12.377008
2,29,1,19,206.938872,511.378635,46.318556,1685,24.20,1000.0,0.019,5.007921,12.375363
3,29,1,21,219.275484,511.092903,44.424332,1550,24.20,1000.0,0.021,5.306467,12.368448
4,29,1,22,227.751666,511.210176,45.848866,1651,24.20,1000.0,0.022,5.511590,12.371286
...,...,...,...,...,...,...,...,...,...,...,...,...
8550,272,3,74,1129.120773,632.701087,45.918239,1656,21.61,1000.0,0.074,24.400300,13.672670
8551,272,3,75,1161.783364,633.420857,45.290053,1611,21.61,1000.0,0.075,25.106139,13.688225
8552,272,3,76,1194.867085,634.314107,45.064588,1595,21.61,1000.0,0.076,25.821078,13.707528
8553,272,3,77,1228.344099,634.962733,45.275994,1610,21.61,1000.0,0.077,26.544516,13.721545


In [43]:
# get time with 0 at start point
def change_time(row):
    row['time'] = row['time'] - improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['time'].min()
    row['del_y'] = row['y'] - improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['y'].min()
    row['del_x'] = row['x'] - improc_df[(improc_df['test']==row['test'])&(improc_df['crop']==row['crop'])]['x'].iloc[0]
    return row

new_improc_df = improc_df.apply(change_time,axis=1)
display(new_improc_df)

,test,crop,frameNum,y_pix,x_pix,diameter_pix,area_pix,pixel_size,record_rate,time,y,x,del_y,del_x
0,29.0,1.0,16.0,193.530266,511.654358,45.862749,1652.0,24.20,1000.0,0.000,4.683432,12.382035,0.000000,0.000000
1,29.0,1.0,17.0,196.702913,511.446602,44.352622,1545.0,24.20,1000.0,0.001,4.760210,12.377008,0.076778,-0.005028
2,29.0,1.0,19.0,206.938872,511.378635,46.318556,1685.0,24.20,1000.0,0.003,5.007921,12.375363,0.324488,-0.006673
3,29.0,1.0,21.0,219.275484,511.092903,44.424332,1550.0,24.20,1000.0,0.005,5.306467,12.368448,0.623034,-0.013587
4,29.0,1.0,22.0,227.751666,511.210176,45.848866,1651.0,24.20,1000.0,0.006,5.511590,12.371286,0.828158,-0.010749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8550,272.0,3.0,74.0,1129.120773,632.701087,45.918239,1656.0,21.61,1000.0,0.052,24.400300,13.672670,23.776469,0.864466
8551,272.0,3.0,75.0,1161.783364,633.420857,45.290053,1611.0,21.61,1000.0,0.053,25.106139,13.688225,24.482308,0.880020
8552,272.0,3.0,76.0,1194.867085,634.314107,45.064588,1595.0,21.61,1000.0,0.054,25.821078,13.707528,25.197247,0.899323
8553,272.0,3.0,77.0,1228.344099,634.962733,45.275994,1610.0,21.61,1000.0,0.055,26.544516,13.721545,25.920686,0.913340


In [44]:
# save results
save_name = 'imgproc_detailed.csv'
new_improc_df.to_csv(os.path.join(imgproc_path,save_name),index=False)